<a href="https://www.kaggle.com/code/klyushnik/improve-a-fixed-model-vol2?scriptVersionId=168375026" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dissolved-oxygen-prediction-in-river-water/train.csv
/kaggle/input/dissolved-oxygen-prediction-in-river-water/test.csv
/kaggle/input/playground-series-s3e21/sample_submission.csv


# load data

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e21/sample_submission.csv")

print('Shape train is', train.shape)

Shape train is (3500, 37)


In [3]:
train.describe()

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
count,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,...,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,8.473834,8.216993,9.291581,9.633064,8.065733,5.671581,9.461363,6.547433,0.340525,...,8.710466,3.002243,3.242654,4.816718,4.754389,4.285660,5.968980,7.837995,3.750349,4.068639
std,1010.507298,1.885712,3.040743,6.817822,1.439039,1.463881,2.721399,1.140365,1.663266,0.284751,...,6.438648,2.918355,2.713923,2.009161,1.547491,2.093388,2.576146,3.724861,1.682421,1.894632
min,0.000000,1.300000,0.000000,0.000000,4.900000,2.300000,0.200000,0.000000,0.000000,0.020000,...,0.044000,0.011000,0.000000,0.570000,0.089000,0.850000,2.025000,2.020000,0.364000,1.600000
25%,874.750000,7.470000,7.100000,7.300000,8.770000,6.830000,4.550000,8.980000,5.877500,0.214500,...,4.450000,1.140000,1.780000,3.520000,3.950000,2.290000,4.950000,5.725000,2.684000,2.900000
50%,1749.500000,8.280000,7.890000,8.300000,9.500000,7.980000,5.800000,9.430000,6.430000,0.247000,...,6.054000,1.730000,1.800000,4.600000,4.800000,3.950000,5.725000,7.500000,3.500000,3.700000
75%,2624.250000,9.110000,9.100000,8.700000,9.545000,9.265000,7.015000,10.070000,7.150000,0.370000,...,14.515000,3.025000,4.237500,5.000000,5.400000,4.725000,6.250000,8.400000,4.150000,4.450000
max,3499.000000,65.930000,46.950000,65.950000,16.900000,21.800000,59.400000,40.190000,15.900000,4.200000,...,35.000000,19.355000,27.025000,11.650000,10.700000,19.800000,55.400000,82.450000,19.375000,10.370000


In [4]:
print('Shape train is', train.shape)

Shape train is (3500, 37)


In [5]:
train.duplicated().sum()

0

# Outlier

In [6]:
from sklearn.linear_model import SGDOneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
svm = SGDOneClassSVM(nu=0.65)
yhat = svm.fit_predict(train.drop('id', axis = 1))

print('Outliers detected (SGD One Class SVM): {}'.format(np.count_nonzero(yhat == -1)))

mask = yhat != -1
data_SVM = train.loc[mask, :].reset_index(drop = True)

iso = IsolationForest(random_state = 2200)
yhat = iso.fit_predict(data_SVM.drop('id', axis = 1))

print('Outliers detected (Isolation Forest): {}'.format(np.count_nonzero(yhat == -1)))

mask = yhat != -1
data_ISO = data_SVM.loc[mask, :].reset_index(drop = True)

lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
yhat = lof.fit_predict(data_ISO.drop(['id', 'target'], axis = 1))

print('Outliers detected (Local Outlier Factor): {}'.format(np.count_nonzero(yhat == -1)))

mask = yhat != -1
train_df = data_ISO.loc[mask, :].reset_index(drop = True)

Outliers detected (SGD One Class SVM): 37
Outliers detected (Isolation Forest): 84
Outliers detected (Local Outlier Factor): 338


In [8]:
print('Shape train is',train_df.shape)

Shape train is (3041, 37)


# Baseline

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
       n_estimators=900,
       max_depth=7,
       n_jobs=-1,
       random_state=42)

In [10]:
X  = train_df.drop(columns = ['id','target'],axis = 1)
y = train_df['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Model 1

In [11]:
rf.fit(X_train, y_train)

feature = pd.DataFrame({'importances': rf.feature_importances_}, 
                   index = X.columns).sort_values(by = 'importances', 
                                                        ascending = False)

In [12]:
feature.head(5)

,importances
O2_1,0.551146
BOD5_5,0.155198
O2_2,0.032353
NO3_2,0.026814
O2_7,0.014090


In [13]:
index_list = list(feature.head(3).index.values)

print('Top feature',index_list)

Top feature ['O2_1', 'BOD5_5', 'O2_2']


In [14]:
list_top = ['id','target']+index_list
for i in train.columns:
    if i not in list_top:
        train_df[i] = 0

train_df.head(5)

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,1,9.10,13.533,40.90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,2,8.21,3.710,5.42,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,3,8.39,8.700,8.10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,4,8.07,8.050,8.65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0


In [15]:
from mlxtend.regressor import StackingCVRegressor
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import optuna
import xgboost as xgb
from optuna.samplers import TPESampler

# Predict

In [16]:
train_df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,1,9.10,13.533,40.900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,2,8.21,3.710,5.420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,3,8.39,8.700,8.100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,4,8.07,8.050,8.650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,3494,8.65,8.367,8.760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3037,3495,8.08,6.250,8.300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.695,0,0
3038,3496,8.09,6.630,6.630,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3039,3497,9.95,8.367,8.433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0


# Submit

In [17]:
train_df.to_csv('submission.csv',index=False)